**This notebook tests pokeapi and combines the pokdex entries from all pokemon games into one enhanced pokedex entry**

In [1]:
import requests

def print_pokedex_entries(pokemon_name):
    url = f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_name.lower()}"
    response = requests.get(url)

    if response.status_code != 200:
        print("Failed to retrieve data.")
        return

    data = response.json()
    entries = data.get("flavor_text_entries", [])

    printed_versions = set()

    for entry in entries:
        language = entry["language"]["name"]
        version = entry["version"]["name"]
        text = entry["flavor_text"].replace('\n', ' ').replace('\f', ' ')

        if language == "en" and version not in printed_versions:
            print(f"{version.capitalize()}: {text}")
            printed_versions.add(version)

# Example usage
print_pokedex_entries("charizard")


Red: Spits fire that is hot enough to melt boulders. Known to cause forest fires unintentionally.
Blue: Spits fire that is hot enough to melt boulders. Known to cause forest fires unintentionally.
Yellow: When expelling a blast of super hot fire, the red flame at the tip of its tail burns more intensely.
Gold: If CHARIZARD be­ comes furious, the flame at the tip of its tail flares up in a whitish- blue color.
Silver: Breathing intense, hot flames, it can melt almost any­ thing. Its breath inflicts terrible pain on enemies.
Crystal: It uses its wings to fly high. The temperature of its fire increases as it gains exper­ ience in battle.
Ruby: CHARIZARD flies around the sky in search of powerful opponents. It breathes fire of such great heat that it melts anything. However, it never turns its fiery breath on any opponent weaker than itself.
Sapphire: CHARIZARD flies around the sky in search of powerful opponents. It breathes fire of such great heat that it melts anything. However, it neve

**Old Enhanced Info**

"Spits fire that is hot enough to melt boulders. Known to cause forest fires unintentionally. When this Pokémon expels a blast of superhot fire, the red flame at the tip of its tail burns more intensely. When expelling a blast of super hot fire, the red flame at the tip of its tail burns more intensely. If [POKEMON] becomes furious, the flame at the tip of its tail flares up in a light blue shade. [POKEMON] flies around the sky in search of powerful opponents. It breathes fire of such great heat that it melts anything. However, it never turns its fiery breath on any opponent weaker than itself. It uses its wings to fly high. The temperature of its fire increases as it gains exper­ ience in battle. When expelling a blast of superhot fire, the red flame at the tip of its tail burns more intensely. It spits fire that is hot enough to melt boulders. It may cause forest fires by blowing flames. If [POKEMON] be­ comes furious, the flame at the tip of its tail flares up in a whitish- blue color. A [POKEMON] flies about in search of strong opponents. It breathes intense flames that can melt any material. However, it will never torch a weaker foe. Breathing intense, hot flames, it can melt almost anything. Its breath inflicts terrible pain on enemies. Its wings can carry this Pokémon close to an altitude of 4,600 feet. It blows out fire at very high temperatures. Breathing intense, hot flames, it can melt almost any­ thing. Its breath inflicts terrible pain on enemies. It is said that [POKEMON]’s fire burns hotter if it has experienced harsh battles. Its wings can carry this POKéMON close to an altitude of 4,600 feet. It blows out fire at very high temperatures."

**New Enhanced info**

"Spits fire that is hot enough to melt boulders. Known to cause forest fires unintentionally. When expelling a blast of super hot fire, the red flame at the tip of its tail burns more intensely. If [POKEMON] be­ comes furious, the flame at the tip of its tail flares up in a whitish- blue color. Breathing intense, hot flames, it can melt almost any­ thing. Its breath inflicts terrible pain on enemies. It uses its wings to fly high. The temperature of its fire increases as it gains exper­ ience in battle. [POKEMON] flies around the sky in search of powerful opponents. It breathes fire of such great heat that it melts anything. However, it never turns its fiery breath on any opponent weaker than itself. A [POKEMON] flies about in search of strong opponents. It breathes intense flames that can melt any material. However, it will never torch a weaker foe. Its wings can carry this POKéMON close to an altitude of 4,600 feet. It blows out fire at very high temperatures. It spits fire that is hot enough to melt boulders. It may cause forest fires by blowing flames. It is said that [POKEMON]’s fire burns hotter if it has experienced harsh battles."

In [ ]:
import requests
import pandas as pd
import re
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# Load sentence transformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to fetch and clean Pokédex entries
def get_enhanced_info(pokemon_name):
    url = f'https://pokeapi.co/api/v2/pokemon-species/{pokemon_name.lower()}'
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None

        data = response.json()
        sentences = []

        for entry in data['flavor_text_entries']:
            if entry['language']['name'] == 'en':
                text = entry['flavor_text'].replace('\n', ' ').replace('\f', ' ')
                
                # Replace Pokémon name with placeholder
                text = re.sub(rf'\b{pokemon_name}\b', '[POKEMON]', text, flags=re.IGNORECASE)
                text = re.sub(' +', ' ', text).strip()

                if text:
                    sentences.append(text)

        # Deduplicate semantically similar sentences
        unique_sentences = []
        if sentences:
            embeddings = model.encode(sentences, convert_to_tensor=True)
            for i, s in enumerate(sentences):
                if all(util.pytorch_cos_sim(embeddings[i], embeddings[j]) < 0.9 for j in range(i)):
                    unique_sentences.append(s)

        return " ".join(unique_sentences)

    except Exception as e:
        print(f"Failed for {pokemon_name}: {e}")
        return None

# Load your cleaned Pokédex
df = pd.read_csv('../Datasets/pokedex_final.csv')

# Apply function with progress
tqdm.pandas(desc="Fetching + Filtering enhanced info")
df['enhanced_info'] = df['name'].progress_apply(get_enhanced_info)

# Save result
output_path = '../Datasets/pokedex_final.csv'
df.to_csv(output_path, index=False)
print(f"✅ Saved enhanced info CSV with filtered entries: {output_path}")


/opt/miniconda3/envs/datascience/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching + Filtering enhanced info: 100%|██████████| 1025/1025 [07:38<00:00,  2.23it/s]


✅ Saved enhanced info CSV with filtered entries: pokedex_final.csv
